In [1]:
# SELECT DISTINCT ?personVal ?famVal ?orgVal ?orgOVal WHERE {
#     GRAPH <http://tbio.orient.cas.cz/chrj_pr> {
#         ?person tbio:isMemberOfFamily ?p .
#         ?p tbio:hasFamily ?fam .
#         FILTER(!sameterm(?fam, tbio:WufengLinFamily)) .
#     }
#     OPTIONAL {
#         ?person ?orgP ?orgEvt .
#         FILTER(sameterm(?orgP, tbio:attendsSchool) || sameterm(?orgP, tbio:hasEmployment) || sameterm(?orgP, tbio:hasMembership))
#         ?orgEvt ?orgAt ?org .
#         FILTER(sameterm(?orgAt, tbio:educationAt) || sameterm(?orgAt, tbio:employmentAt) || sameterm(?orgAt, tbio:hasMembershipAt))
#         ?org rdf:type ?orgO .
#     }
#     BIND(STR(?person) AS ?personStr) .
#     BIND(REPLACE(?personStr, "http://tbio.orient.cas.cz#", "") AS ?personVal) .
#     BIND(STR(?fam) AS ?famStr) .
#     BIND(REPLACE(?famStr, "http://tbio.orient.cas.cz#", "") AS ?famVal) .
#     BIND(STR(?org) AS ?orgStr) .
#     BIND(REPLACE(?orgStr, "http://tbio.orient.cas.cz#", "") AS ?orgVal) .
#     BIND(STR(?orgO) AS ?orgOStr) .
#     BIND(REPLACE(?orgOStr, "http://tbio.orient.cas.cz#", "") AS ?orgOVal) .
# }

In [2]:
import pandas as pd

In [3]:
filepath = '/Volumes/backup_128G/z_repository/TBIO_data/RequestsFromTana/20190515'

filename = 'personFamilyOrgPeriod-20190516.tsv'

read_filename = '{0}/{1}'.format(filepath, filename)

In [4]:
workDf = pd.read_csv(read_filename, delimiter='\t')
workDf = workDf.dropna().reset_index(drop=True)
workDf.shape, workDf.head()

((8461, 4),   ?personVal ?famVal     ?orgVal                      ?orgOVal
 0        林熊祥    嚴復家系     林本源製糖會社  JapaneseColonialOrganization
 1        林熊祥    嚴復家系     林本源製糖會社               PrivateBusiness
 2        林熊祥    嚴復家系  台灣大有物產株式會社  JapaneseColonialOrganization
 3        林熊祥    嚴復家系  台灣大有物產株式會社               PrivateBusiness
 4        林熊祥    嚴復家系    台北商事株式會社  JapaneseColonialOrganization)

In [5]:
# SELECT DISTINCT ?s WHERE {
#     GRAPH ?graph {
#         ?s rdfs:subClassOf tbio:Division_Period .
#     }
# }

Division_Period = [
    'JapaneseColonialOrganization',
    'EarlyPostWarOrganization',
    'RepublicanOrganization',
    'BothPeriods',
    'PreRetrossecionRepublicanOrganization',
    'QingPeriodOrganization',
    'UnknownPeriodOrganization'
]

In [6]:
# SELECT DISTINCT ?s WHERE {
#     GRAPH ?graph {
#         ?s rdfs:subClassOf tbio:SocietalSector .
#     }
# }

SocietalSector = [
    'Police',
    'ArmedForces',
    'Hospital',
    'ReligiousOrganization',
    'MassMedia',
    'PublicAdministration',
    'PrivateBusiness',
    'VoluntaryAssociation',
    'Academia_Education',
    'StateBusiness',
    'CulturalOrganization',
    'Politics',
    'UnknownSector'    
]

In [7]:
workList = []
for idx in range(0, len(workDf)):
    row = workDf.loc[idx]

    person = str(row['?personVal'])
    family = str(row['?famVal'])
    org = str(row['?orgVal'])
    
    if org == '':
        continue
    orgO = str(row['?orgOVal'])

    workList.append([person, family, org, orgO])
    
print(len(workList))

8461


# Read person-family map table

In [8]:
familymembers = 'Familymembers.xlsx'

read_familymembers = '{0}/{1}'.format(filepath, familymembers)

In [9]:
fmDf = pd.read_excel(read_familymembers)
fmDf.shape, fmDf.head()

((4546, 2),   personStr familyStr
 0       何佩娟      何傳家系
 1       何佩珊      何傳家系
 2       何佩珍      何傳家系
 3        何傳      何傳家系
 4       何培庭      何傳家系)

In [10]:
fmDic = {}
for idx in range(0, len(fmDf)):
    row = fmDf.loc[idx]

    person = str(row['personStr'])
    family = str(row['familyStr'])
    
    if person not in fmDic:
        fmDic[person] = family
    elif family != fmDic[person]:
        print("Dup:", person, family, fmDic[person])

# fmDic

In [11]:
def getPeriod(INperson, INfamily, INorg):
    for row in workList:
        person = row[0]
        family = row[1]
        org = row[2]
        orgO = row[3]

        if person == INperson and family == INfamily and org == INorg:
            if orgO in Division_Period:
                return orgO
    return ''

def getSocietal(INperson, INfamily, INorg):
    for row in workList:
        person = row[0]
        family = row[1]
        org = row[2]
        orgO = row[3]

        if person == INperson and family == INfamily and org == INorg:
            if orgO in SocietalSector:
                return orgO
    return ''

def getFamilyName(INperson):
    if INperson not in fmDic:
        print(INperson, " Not found!")
        return ''
    return fmDic[INperson]

resList = []
for row in workList:
    person = row[0]
    family = row[1]
    org = row[2]
    
    period = getPeriod(person, family, org)
    if period == '':
        continue
        
    societal = getSocietal(person, family, org)
    if societal == '':
        continue

    familyName = getFamilyName(person)
    if familyName == '':
        continue
        
    resList.append([person, familyName, org, societal, period])
    
print(len(resList), resList[0:5])

7174 [['林熊祥', '林維讓家系', '林本源製糖會社', 'PrivateBusiness', 'JapaneseColonialOrganization'], ['林熊祥', '林維讓家系', '林本源製糖會社', 'PrivateBusiness', 'JapaneseColonialOrganization'], ['林熊祥', '林維讓家系', '台灣大有物產株式會社', 'PrivateBusiness', 'JapaneseColonialOrganization'], ['林熊祥', '林維讓家系', '台灣大有物產株式會社', 'PrivateBusiness', 'JapaneseColonialOrganization'], ['林熊祥', '林維讓家系', '台北商事株式會社', 'PrivateBusiness', 'JapaneseColonialOrganization']]


In [12]:
orgDf = pd.DataFrame(resList, columns=['Person', 'FamilyName', 'Organization', 'SocietalSector', 'OrganizationPeriod'])
orgDf.drop_duplicates(keep='first', inplace=True)
orgDf.sort_values(by=['FamilyName', 'Person'], inplace=True)
orgDf.head()

,Person,FamilyName,Organization,SocietalSector,OrganizationPeriod
3813,何傳,何傳家系,高雄市,PublicAdministration,BothPeriods
3815,何傳,何傳家系,高雄州,PublicAdministration,JapaneseColonialOrganization
3817,何傳,何傳家系,民報商事社,PrivateBusiness,JapaneseColonialOrganization
3819,何傳,何傳家系,台灣木工株式會社,PrivateBusiness,JapaneseColonialOrganization
3821,何傳,何傳家系,永豐商店理事會,PrivateBusiness,JapaneseColonialOrganization


In [13]:
write_file_to = '{0}/{1}'.format(filepath, 'personFamilyOrgPeriod_20190516_v2.xlsx')
orgDf.to_excel(write_file_to, index=False)